In [ ]:
import ipywidgets as widgets
import datetime

In [ ]:
import webapp.poissonmagique.models as pmm

In [ ]:
q=pmm.Queue(name="main",maildir="/tmp")
q.save()

In [ ]:
mid1 = pmm.MessageID(message_id="message1",key="message1",queue=q)
mid1.save()

In [ ]:
mid2 = pmm.MessageID(message_id="message2",key="message2",queue=q)
mid2.save()
mid3 = pmm.MessageID(message_id="message3",key="message3",queue=q)
mid3.save()

In [ ]:
gm=pmm.Human.objects.all()[0]

In [ ]:
user1 = User.objects.create_user('user1','user1@example.com','secret1')
user1.save()
user2 = User.objects.create_user('user2','user2@example.com','secret2')
user2.save()

In [ ]:
h1 = pmm.Human(user=user1, name=user1.username, mail_address=user1.email)
h1.save()
h2 = pmm.Human(user=user2, name=user2.username, mail_address=user2.email)
h2.save()

In [ ]:
campaign=pmm.Campaign.objects.all()[0]

In [ ]:
char1=pmm.Character(name="PC1",controller=h1,mail_address="pc1@poissonmagique.net",campaign=campaign)
char1.save()
char2=pmm.Character(name="PC2",controller=h2,mail_address="pc2@poissonmagique.net",campaign=campaign)
char2.save()
char3=pmm.Character(name="NPC1",controller=gm,mail_address="npc1@poissonmagique.net",campaign=campaign)
char3.save()
char4=pmm.Character(name="NPC2",controller=gm,mail_address="npc2@poissonmagique.net",campaign=campaign)
char4.save()

In [ ]:
m1 = pmm.Message(message_id="mid1",subject="At MtlPy",text="""When I get to MtlyPy I say hi.
I try to find a chair as close to the beer as possibe.
If anybody sees me, I try to look I'm PEP-8 complaint.
""",author_human=h1,author_character=char1,when=datetime.datetime.now(),campaign=campaign)
m1.save()

In [ ]:
m2 = pmm.Message(message_id="mid2",subject="At MtlPy",text="""I came earlier at MtlyPy and drank all the beer.
I think nobody realized I am a Java dev yet!
""",author_human=h2,author_character=char2,when=datetime.datetime.now(),campaign=campaign)
m2.save()

In [ ]:
m3 = pmm.Message(message_id="mid3",subject="Unrelated email",text="""
Welcome to Zork!

Dungeon is a game of adventure, danger, and low cunning. In it
you will explore some of the most amazing territory ever seen by mortal
man. Hardened adventurers have run screaming from the terrors contained
within.
""",author_human=h2,author_character=char2,when=datetime.datetime.now(),campaign=campaign)
m3.save()

In [ ]:
def multireply(campaign):
    
    all_sources = []
    selected_sources = []
    response = [ """Type your response here""" ]
    sent = [ False ]
    
    for mail in Message.objects.filter(campaign=campaign):
        all_sources.append((str(mail),mail.message_id))
        
    all_mails = widgets.Select(description="sources",options=all_sources)
    
    def add_source(source):
        selected_sources.append(all_mails.value)
        make_control()
        
    all_mails.on_trait_change(add_source, 'value')
    
    box = widgets.VBox()
    def make_control():
        sources_children = []
        for msgid in selected_sources:
            msg = pmm.Message.objects.filter(message_id = msgid)[0]
            sources_children.append(widgets.HTML("""<h2>%s</h2><p>%s</p><p>&nbsp;</p>""" % (msg.subject,msg.text)))
        sources = widgets.VBox(children=sources_children)
        send = widgets.Button(description="send" if not sent[0] else "sent",disabled=sent[0])
        email = widgets.Textarea(description="Response",value=response[0],width=500,height=120)
        def text_edit(source):
            response[0] = email.value
        email.on_trait_change(text_edit, 'value')
        
        def send_email(source):
            print selected_sources, response[0]
            sent[0] = True
            make_control()
        send.on_click(send_email)
        
        box.children = (all_mails,sources,email,send)
        
    make_control()
    return box

In [ ]:
multireply(campaign)